In [6]:
from ETL2 import fetch_data_from_sqlite
from datetime import date
import pandas as pd

In [7]:
date_ = date(2023, 5, 12)
results = fetch_data_from_sqlite(date_=date_)

In [8]:
df = pd.DataFrame(results)

In [16]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):

    df["Division"] = df.ballot_meta.apply(lambda x: x["division_number_integer"])
    df["Division"] = df["Division"].astype("int64")

    # Drop column: 'ballot_meta'
    df = df.drop(columns=['ballot_meta'])

    # Performed 2 aggregations grouped on columns: 'province', 'amphur', 'tambon'
    df = df.groupby(['province', 'amphur', 'tambon']).agg(ballot_units_last=('ballot_units', 'last'), Division_last=('Division', 'last')).reset_index()

    df.ballot_units_last = df.ballot_units_last.apply(pd.DataFrame)
    def add_parent_column(x):
        x.ballot_units_last["Province"] = x.province
        x.ballot_units_last["Amphur"] = x.amphur
        x.ballot_units_last["Tambon"] = x.tambon
        return x
    
    df.apply(add_parent_column, axis=1)

    df = pd.concat(df.ballot_units_last.to_list())

    df.drop(columns=["unit_number"], inplace=True)

    # rename unit_number_integer to unit_number
    df.rename(columns={"unit_number_integer": "unit_number", "id": "vote62_id"}, inplace=True)
    df.total_volunteers = df.total_volunteers.fillna('0')

    df.total_volunteers = df.total_volunteers.astype("int64")

    df.unit_info = df.unit_info.str.strip()

    # make title_case
    df.columns = df.columns.str.title()
    
    df.rename(columns={"Unit_Info": "Unit_Street_Address"}, inplace=True)
    
    
    return df

df_clean = clean_data(df.copy())
df_clean.to_parquet(f"vote62_{date_.strftime('%Y%m%d')}.parquet")
df_clean.head()

,Division_Number,Vote62_Id,Is_Verify,Total_Volunteers,Unit_Street_Address,Unit_Number,Province,Amphur,Tambon
0,3,39001,False,0,,1,กระบี่,คลองท่อม,คลองท่อมเหนือ
1,3,39002,False,0,,2,กระบี่,คลองท่อม,คลองท่อมเหนือ
2,3,39003,False,0,,3,กระบี่,คลองท่อม,คลองท่อมเหนือ
3,3,39004,False,0,,4,กระบี่,คลองท่อม,คลองท่อมเหนือ
4,3,39005,False,0,,5,กระบี่,คลองท่อม,คลองท่อมเหนือ
